# Corona Trends in Denmark, Sweden and Switzerland

The following tables and figures display Covid-19 data for Denmark, Sweden and Switzerland. Data is taken from the John Hopkins University [github repository](https://github.com/CSSEGISandData/COVID-19). The dynamics for these countries clearly differ...

In [62]:
from IPython.display import HTML

In [63]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [19]:
# jupyter nbconvert --to html --TemplateExporter.exclude_input=True --no-prompt Denmark_Sweden_Switzerland.ipynb
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.signal import savgol_filter

from corona import DataDownloader
from corona import apply_to_column as a2c

In [3]:
population = pd.Series(name='population',data=
    {'Denmark':5806000,
     'Sweden':10230000,
     'Switzerland':8570000})

In [4]:
countries = ['Switzerland','Sweden','Denmark']

# load data and aggregate over states (elements of countries)
df = (
    DataDownloader().read_jhu()
    .query('country in @countries')
    .groupby(['country','serie','date'])
    .sum()
    .reset_index() 
    .sort_values(by=['country','serie','date']) )

In [5]:
# calculate number per 1 million
df['cumulative1mio'] = df.cumulative / population.reindex(df.country).values * 1e6
df['daily1mio'] = df.daily / population.reindex(df.country).values * 1e6

## Statistics

A few overall numbers. Observations
- Sweden has the most deaths, both in absolute numbers and per million inhabitants
- The ratio of confirmed to deaths cases is double in Sweden 

In [6]:
statdf = (
    df.groupby(['country','serie'])[['cumulative']]
    .max()
    .dropna()
    .astype(int)
    .reset_index()
    .pivot(index='country',columns='serie',values='cumulative')
    .sort_values('deaths',ascending=False)  
    .join(population)
    .assign(confirmed_1mio=lambda df: df.confirmed / df.population * 1e6)
    .assign(deaths_1mio=lambda df: df.deaths / df.population * 1e6)
    .astype(int)
)

statdf['deaths_confirmed_percent'] =  np.round(statdf.deaths/statdf.confirmed *100,1)
statdf.head()

,confirmed,deaths,population,confirmed_1mio,deaths_1mio,deaths_confirmed_percent
country,,,,,,
Sweden,23216,2854,10230000,2269,278,12.3
Switzerland,30009,1795,8570000,3501,209,6.0
Denmark,10019,503,5806000,1725,86,5.0


## Daily cases

Trend lines are running averages over 1 week in order to smooth out variations due to reporting breaks on weekends.

**Note: All plots are interactive. You can zoom in...**

### Total per country

In [7]:
def trend(s):
    if len(s) >= 7:
        out = s.replace({0:1}).rolling(7,center=True,win_type='boxcar',min_periods=1).mean()
        return pd.Series(out, index=s.index)
    else:
        return s * np.nan

df['daily_trend'] = a2c(df,'daily',trend)
df['daily1mio_trend'] = a2c(df,'daily1mio',trend)

In [24]:
for serie in df.serie.unique():
    sel = df.serie == serie
    #tmp = df.query("country==@country",)

    fig = px.bar(df[sel],x='date',y='daily',color='country',log_y=False,barmode='group',
                 title='total number of ' + serie + ' cases')
    #fig.update_yaxes(matches=None)
    for i,country in enumerate(df.country.unique()):
        fig.add_scatter(x=df[sel].query('country==@country').date, y=df[sel].query('country==@country').daily_trend, 
                        mode='lines',line={'color':{0:'blue',1:'red',2:'green'}[i]}, 
                        name = country + ' weekly average')
    fig.update_layout(legend=dict(x=0.05, y=0.9))
    fig.show()

### Per 1 million inhabitants

Sweden has currently around 6 deaths per day and 1 million inhabitant. In Switzerland and Denmark the same number is around 2.  

In [25]:
for serie in df.serie.unique():
    sel = df.serie == serie
    #tmp = df.query("country==@country",)

    fig = px.bar(df[sel],x='date',y='daily',color='country',log_y=False,barmode='group',
                 title= serie + ' cases per 1 million inhabitants')
    #fig.update_yaxes(matches=None)
    for i,country in enumerate(df.country.unique()):
        fig.add_scatter(x=df[sel].query('country==@country').date, y=df[sel].query('country==@country').daily_trend, 
                        mode='lines',line={'color':{0:'blue',1:'red',2:'green'}[i]}, 
                        name = country + ' weekly average')
    fig.update_layout(legend=dict(x=0.05, y=0.9))
    fig.show()